# Epidemic simulation on a graph

In [1]:
from simulate import simulate

## Epidemic on a known graph

We tackle the first problem, which consists in simulating an epidemic on a symmetric $k$-regular undirected graph with $|\mathcal{V}|=500$ nodes and $k=4$ for $15$ weeks (steps).

Firstly, we define a function to generate a $k$-regular undirected graph $\mathcal{G}=(\mathcal{V}, \mathcal{E})$ with $k=4$ and $500$ nodes. Every node is connected to the $k=4$ nodes whose index is closest to their own modulo $n$. The graph is unweighted and undirected.